<a href="https://colab.research.google.com/github/Aminfaraji/paddleocr_persian/blob/main/paddleocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !zip -r "/content/drive/MyDrive/paddel-ocr-shotor.zip" "/content/drive/MyDrive/paddel-ocr-shotor"

**Install Paddleocr compatible with the system (cuda11.2 & system linux)**


**download pretrained model this link**

https://drive.google.com/file/d/18wR7X2zDEdWBHBTPXSJT5S5L1_O7rmKh/view?usp=sharing

In [ ]:
!python -m pip install paddlepaddle-gpu==2.4.1.post112 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
%cd /content/PaddleOCR
!pip install -r requirements.txt

In [ ]:
!mkdir pretrained_model
%cd pretrained_model/
!wget https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/arabic_PP-OCRv3_rec_train.tar
!tar -xf arabic_PP-OCRv3_rec_train.tar && rm -rf arabic_PP-OCRv3_rec_train.tar

**download dataset shotor from google drive**

https://drive.google.com/file/d/1kj8wQ45R63Gyp-Qv2mJaa1C4Y0u1JgEC/view?usp=sharing, https://drive.google.com/file/d/1zrZUJjeJgz0iAVeK8QmMZVAUdPDSpman/view?usp=sharing

In [ ]:
!mkdir /content/PaddleOCR/train_data
!unzip /content/drive/MyDrive/MY_dataset.zip -d /content/PaddleOCR/train_data
# !unzip /content/drive/MyDrive/IDPL_train.zip -d /content/PaddleOCR/train_data
# !unzip /content/drive/MyDrive/IDPL_eval.zip -d /content/PaddleOCR/train_data

In [5]:
import glob,pandas,shutil
# a=glob.glob("/content/PaddleOCR/train_data/en_train_filtered/*.csv")
# shutil.move(a[0],'/content/PaddleOCR/train_data')
#a=glob.glob("/content/PaddleOCR/train_data/en_val/*.csv")
#shutil.move(a[0],'/content/PaddleOCR/train_data')

**convert file .csv to .txt for train Paddleocr. (Paddleocr support of format .txt for train model)**

**recommend download file .txt**
https://drive.google.com/file/d/1-15OsG6mAxBgVXtjCE2bKVKk5Ry_Dcr3/view?usp=sharing,
*****************************
https://drive.google.com/file/d/1Q4FuTYAKjWtUgGBsAK0blYuSQm8Rme99/view?usp=sharing

In [ ]:
# !rm -rf /content/PaddleOCR/train_data/en_train_filtered/labels.csv
# !rm -rf /content/PaddleOCR/train_data/en_val/labels.csv
# !cp /content/drive/MyDrive/val.txt -r /content/PaddleOCR/train_data/
# !cp /content/drive/MyDrive/train.txt -r /content/PaddleOCR/train_data/

In [ ]:
# # # Read in the file
# # with open('/content/drive/MyDrive/val.txt', 'r') as file :
# #   filedata = file.read()

# # # Replace the target string
# # filedata = filedata.replace('.jpg', '.tif')

# # # Write the file out again
# # with open('/content/drive/MyDrive/val.txt', 'w') as file:
# #   file.write(filedata)

# # # Read in the file
# # with open('/content/drive/MyDrive/train.txt', 'r') as file :
# #   filedata = file.read()

# # # Replace the target string
# # filedata = filedata.replace('.jpg', '.tif')

# # # Write the file out again
# # with open('/content/drive/MyDrive/train.txt', 'w') as file:
# #   file.write(filedata)
# # 1. Open the CSV file in reading mode and the TXT file in writing mode
# with open('/content/PaddleOCR/train_data/train.csv', 'r') as f_in, open('/content/PaddleOCR/train_data/train.txt', 'w') as f_out:

#     # 2. Read the CSV file and store in variable
#     content = f_in.read()

#     # 3. Write the content into the TXT file
#     f_out.write(content)
# !rm -rf /content/PaddleOCR/train_data/train.csv
# with open(r"/content/PaddleOCR/train_data/train.txt", 'r+') as fp:
#     # read an store all lines into list
#     lines = fp.readlines()
#     # move file pointer to the beginning of a file
#     fp.seek(0)
#     # truncate the file
#     fp.truncate()

#     # start writing lines
#     # iterate line and line number
#     for number, line in enumerate(lines):
#         # delete line number 5 and 8
#         # note: list index start from 0
#         if number not in [0]:
#             fp.write(line)


In [6]:
import cv2 
import os 
#from imutils import perspective
import numpy as np 
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from imutils import perspective

**continue trained model**

In [1]:
%cd /content/PaddleOCR

/content/PaddleOCR


In [ ]:
#!rm -rf /content/drive/MyDrive/output_IDPL

In [ ]:
!python3 tools/train.py -c /content/PaddleOCR/pretrained_model/arabic_PP-OCRv3_rec_train/config.yml -o Global.pretrained_model=/content/PaddleOCR/pretrained_model/arabic_PP-OCRv3_rec_train/best_accuracy

In [ ]:
!python3 tools/train.py -c /content/drive/MyDrive/output_IDPL-2/config.yml -o Global.checkpoints=/content/drive/MyDrive/output_IDPL-2/latest

In [ ]:
files=glob.glob("/content/drive/MyDrive/output_IDPL/*")
import os
for file in files:
  if file.find('230')<=0 and file.find('latest')<=0 and file.find('train')<=0 and file.find('best')<=0 and file.find('config')<=0:
    os.remove(file)

**evaluate dataset shotor**

In [ ]:
!python3 -m paddle.distributed.launch --gpus '0' tools/eval.py -c /content/drive/MyDrive/output/v3_arabic_mobile/config.yml -o Global.checkpoints=/content/drive/MyDrive/output/v3_arabic_mobile/best_accuracy

In [ ]:
%cd ..

/content/PaddleOCR


**test image uploaded on content**

In [4]:
!python3 tools/infer_rec.py -c /content/drive/MyDrive/output_my_dataset/v3_arabic_mobile/config.yml -o Global.pretrained_model=/content/drive/MyDrive/output_my_dataset/v3_arabic_mobile/best_accuracy  Global.infer_img=/content/Amin_35993.png

[2023/02/17 21:21:42] ppocr INFO: Architecture : 
[2023/02/17 21:21:42] ppocr INFO:     Backbone : 
[2023/02/17 21:21:42] ppocr INFO:         last_conv_stride : [1, 2]
[2023/02/17 21:21:42] ppocr INFO:         last_pool_type : avg
[2023/02/17 21:21:42] ppocr INFO:         name : MobileNetV1Enhance
[2023/02/17 21:21:42] ppocr INFO:         scale : 0.5
[2023/02/17 21:21:42] ppocr INFO:     Head : 
[2023/02/17 21:21:42] ppocr INFO:         head_list : 
[2023/02/17 21:21:42] ppocr INFO:             CTCHead : 
[2023/02/17 21:21:42] ppocr INFO:                 Head : 
[2023/02/17 21:21:42] ppocr INFO:                     fc_decay : 1e-05
[2023/02/17 21:21:42] ppocr INFO:                 Neck : 
[2023/02/17 21:21:42] ppocr INFO:                     depth : 2
[2023/02/17 21:21:42] ppocr INFO:                     dims : 64
[2023/02/17 21:21:42] ppocr INFO:                     hidden_dims : 120
[2023/02/17 21:21:42] ppocr INFO:                     name : svtr
[2023/02/17 21:21:42] ppocr INFO:   

**Recommend:**

Please change the (configuration) file for train or evaluation.